In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Drug;
DROP TABLE IF EXISTS Pathology;
DROP TABLE IF EXISTS DrugUse;

CREATE TABLE Drug (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/drug.csv');

CREATE TABLE Pathology (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/pathology.csv');

CREATE TABLE DrugUse (
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  codeDrug INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(codeDrug)
    REFERENCES Drug(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    idPerson,
    codePathology,
    codeDrug
FROM CSVREAD('../../../data/faers-2017/drug-use.csv');

CREATE TABLE SideEffect(
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT idPerson, codePathology
FROM CSVREAD('../../../data/faers-2017/sideeffect.csv');

In [3]:
SELECT * FROM Drug;
SELECT * FROM Pathology;
SELECT * FROM DrugUse;
SELECT * FROM SideEffect;

# FDA Adverse Event Reporting System (FAERS)

Considere a base de dados que reúne relatos de efeitos adversos de drogas: [FDA Adverse Event Reporting System (FAERS)](https://open.fda.gov/data/faers/).

As fontes de dados são publicadas e documentadas em: [Latest Quarterly Data Files](https://www.fda.gov/drugs/fda-adverse-event-reporting-system-faers/fda-adverse-event-reporting-system-faers-latest-quarterly-data-files)

# Interação entre medicamentos
Considere como uma interação toda vez que um medicamento A é tomado em conjunto com um medicamento B por uma pessoa. Crie uma consulta que retorne para cada par de medicamento A,B o número de interações, somente para aqueles medicamentos que são tomados em conjunto mais de 30 vezes (passaremos a chamar de **grupo mais 30**).

In [ ]:
CREATE VIEW InteractionsCount AS
SELECT A, B, count(*) Quantity
FROM (
    SELECT D1.codeDrug AS A, D2.codeDrug AS B
    FROM DrugUse AS D1, DrugUse AS D2
    WHERE D1.idPerson = D2.idPerson AND D1.codeDrug != D2.codeDrug
)
GROUP BY A, B;

SELECT A, B, Quantity FROM InteractionsCount WHERE Quantity > 30;

## Tabela Resultante

Se estiver demorando muito para gerar a tabela, você poderá usar a seguinte tabela resultante pronta. Essa tabela tem todas as interações, você ainda precisará filtrar o **grupo mais 30**.

In [4]:
CREATE TABLE DrugInteraction (
  druga INT NOT NULL,
  drugb INT NOT NULL,
  weight INT NOT NULL,
  PRIMARY KEY(druga, drugb)
) AS SELECT
    druga,
    drugb,
    weight
FROM CSVREAD('../../../data/faers-2017/results/drug-interaction.csv');

SELECT * FROM DrugInteraction;

# Suporte

### Qual o suporte que cada uma das associações tem?

Gere uma consulta que apresente o suporte do **grupo mais 30**. Ordene o resultado por ordem de suporte. 

Observação: Para que o resultado da divisão seja fracionária, converta o numerador em `DOUBLE` usando:
~~~sql
CAST(campo AS DOUBLE)
~~~

In [5]:
DROP VIEW IF EXISTS WeightSUM;

CREATE VIEW WeightSUM AS
SELECT SUM(weight) AS sum
FROM DrugInteraction;

SELECT DI.druga, DI.drugb, weight,  CAST(DI.weight AS DOUBLE)/CAST(WS.sum AS DOUBLE) AS Support
FROM DrugInteraction AS DI, WeightSUM AS WS
WHERE DI.weight > 30
ORDER BY Support DESC;

# Confiança

### Qual a confiança que cada uma das associações tem?

Gere uma consulta que apresente a confiança do **grupo mais 30**. Ordene o resultado por ordem de confiança.

In [6]:
DROP VIEW IF EXISTS DrugACount;

CREATE VIEW DrugACount AS
SELECT druga, Sum(weight) AS NumberOFTotalInterations 
FROM (
    SELECT druga, weight FROM DrugInteraction
    UNION ALL
    SELECT drugb AS druga, weight FROM DrugInteraction)

GROUP BY druga
ORDER BY druga;

SELECT 
    DI.druga, 
    DI.drugb,
    DI.weight, 
    CAST (DI.weight AS DOUBLE)/CAST(DAC.NumberOFTotalInterations AS DOUBLE) as Confidence
FROM DrugInteraction AS DI, DrugACount AS DAC
WHERE DI.weight > 30 and DI.drugA = DAC.druga
GROUP BY DI.druga, DI.drugb
ORDER BY Confidence DESC;

# Efeito Colateral

Para cada medicamento, qual o percentual do medicamento A que tem efeito colateral junto com o medicamento B, quando comparado com seu uso sem o medicamento B.